# Logistic Regression

In [256]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

import os
import warnings
warnings.filterwarnings('ignore')

In [257]:
#importing the dataset
df=pd.read_csv('tourist sentiment.csv')
print(df.shape)
df

(7102, 5)


,ID,location,review,sentiment,sentiment_review
0,1,Syambhunath,It is at the top of valleys mountain. Best pl...,1,positive
1,2,Syambhunath,This place has a significant importance in Bud...,1,positive
2,3,Syambhunath,Visited this from the other side on a rainy ev...,1,positive
3,4,Syambhunath,A beautiful temple situated in the capital wit...,1,positive
4,5,Syambhunath,"great, beautiful, historic & religious place.....",1,positive
...,...,...,...,...,...
7097,7267,Pokhara,"It's a nice place to sit back, and enjoy. The ...",1,positive
7098,7268,Pokhara,"Excellent Place to visit, Lifetime memories",1,positive
7099,7269,Pokhara,It's very photogenic and relaxing when there a...,1,positive
7100,7270,Pokhara,U can get real definition of nature's beauty a...,1,positive


Exploratery data analysis



In [258]:
#df.drop(df.loc[df['sentiment']==0].index, inplace=True)
df.drop(df.index[df['sentiment'] == 0], inplace = True)

In [259]:
#Summary of the dataset
df.describe()

,ID,sentiment
count,5401.000000,5401.000000
mean,3606.660804,0.920755
std,2155.914768,0.390176
min,1.000000,-1.000000
25%,1787.000000,1.000000
50%,3467.000000,1.000000
75%,5562.000000,1.000000
max,7271.000000,1.000000


Sentiment count



In [260]:
#sentiment count
df['sentiment'].value_counts()

 1    5187
-1     214
Name: sentiment, dtype: int64

In [262]:
df

,ID,location,review,sentiment,sentiment_review
0,1,Syambhunath,It is at the top of valleys mountain. Best pl...,1,positive
1,2,Syambhunath,This place has a significant importance in Bud...,1,positive
2,3,Syambhunath,Visited this from the other side on a rainy ev...,1,positive
3,4,Syambhunath,A beautiful temple situated in the capital wit...,1,positive
4,5,Syambhunath,"great, beautiful, historic & religious place.....",1,positive
...,...,...,...,...,...
7097,7267,Pokhara,"It's a nice place to sit back, and enjoy. The ...",1,positive
7098,7268,Pokhara,"Excellent Place to visit, Lifetime memories",1,positive
7099,7269,Pokhara,It's very photogenic and relaxing when there a...,1,positive
7100,7270,Pokhara,U can get real definition of nature's beauty a...,1,positive


In [263]:
#df.to_csv('tourist sentiment positive and negative.csv', index=False)


In [264]:
df.drop(columns = ['ID', 'location', 'sentiment'], axis=1, inplace=True)


Spliting the training dataset



In [265]:
#split the dataset  
#train dataset
train_reviews=df.review[:4000]
train_sentiments=df.sentiment_review[:4000]
#test dataset
test_reviews=df.review[4000:]
test_sentiments=df.sentiment_review[4000:]
print(train_reviews.shape,train_sentiments.shape)
print(test_reviews.shape,test_sentiments.shape)


(4000,) (4000,)
(1401,) (1401,)


Text normalization



In [266]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

Removing html strips and noise text



In [267]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
df['review']=df['review'].apply(denoise_text)

Removing special characters



In [268]:
#Define function for removing special characters
def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text
#Apply function on review column
df['review']=df['review'].apply(remove_special_characters)

Text stemming



In [269]:
#Stemming the text
def simple_stemmer(text):
    ps=nltk.porter.PorterStemmer()
    text= ' '.join([ps.stem(word) for word in text.split()])
    return text
#Apply function on review column
df['review']=df['review'].apply(simple_stemmer)

Removing stopwords



In [270]:
#set stopwords to english
stop=set(stopwords.words('english'))
print(stop)

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
df['review']=df['review'].apply(remove_stopwords)


{'there', 'if', 'more', 'so', 'were', "mightn't", 'now', 'd', "didn't", 'hasn', 'into', 'we', 'most', 'down', 'own', 'that', 'who', 'o', 'here', 'other', 'their', 'for', 're', 'be', 'once', 'wasn', 'my', 'himself', 'being', 'very', 'mightn', 'no', 'did', 'while', 'should', 'am', 'couldn', 'from', 'this', 'yourself', 's', 'with', 'by', 'these', 'not', 'his', 'was', 'until', 'your', 'weren', "she's", 'i', 'as', "you'd", 'yours', 'further', 'didn', 'is', 've', 'ma', 'when', 'had', "wouldn't", 'or', 'in', 'just', 'then', 'are', 'where', 'all', "aren't", 'y', 'them', 'those', 'myself', 'and', 'me', 'under', 'don', 'both', "wasn't", 'before', 'few', 'does', 'they', 'about', 'theirs', 'have', 'up', 'doesn', "isn't", 'of', 'on', 'aren', 'the', "weren't", 'any', 'after', "hasn't", 'how', "needn't", 'can', 'which', 'herself', "you're", 't', 'isn', "that'll", 'each', 'why', "haven't", 'out', 'between', 'm', "hadn't", 'whom', 'our', 'its', 'won', 'against', 'itself', 'hadn', "couldn't", 'been', 'a

Normalized train reviews



In [271]:
#normalized train reviews
norm_train_reviews=df.review[:4000]
norm_train_reviews[0]
#convert dataframe to string
#norm_train_string=norm_train_reviews.to_string()
#Spelling correction using Textblob
#norm_train_spelling=TextBlob(norm_train_string)
#norm_train_spelling.correct()
#Tokenization using Textblob
#norm_train_words=norm_train_spelling.words
#norm_train_words

'top valley mountain best place get pleasur realli love place see whole kathmandu valley best visit onc life must visit life get best experi life'

**Normalized test reviews**



In [272]:
#Normalized test reviews
norm_test_reviews=df.review[4000:]
#norm_test_reviews[]
##convert dataframe to string
#norm_test_string=norm_test_reviews.to_string()
#spelling correction using Textblob
#norm_test_spelling=TextBlob(norm_test_string)
#print(norm_test_spelling.correct())
#Tokenization using Textblob
#norm_test_words=norm_test_spelling.words
#norm_test_words

**Bags of words model**



In [273]:
#Count vectorizer for bag of words
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
#transformed train reviews
cv_train_reviews=cv.fit_transform(norm_train_reviews)
#transformed test reviews
cv_test_reviews=cv.transform(norm_test_reviews)

print('BOW_cv_train:',cv_train_reviews.shape)
print('BOW_cv_test:',cv_test_reviews.shape)
#vocab=cv.get_feature_names()-toget feature names

BOW_cv_train: (4000, 54392)
BOW_cv_test: (1401, 54392)


**Term Frequency-Inverse Document Frequency model (TFIDF)**





It is used to convert text documents to matrix of tfidf features.


In [274]:
#Tfidf vectorizer
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(norm_train_reviews)
#transformed test reviews
tv_test_reviews=tv.transform(norm_test_reviews)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)

Tfidf_train: (4000, 54392)
Tfidf_test: (1401, 54392)


**Labeling the sentiment text**



In [275]:
#labeling the sentient data
lb=LabelBinarizer()
#transformed sentiment data
sentiment_data=lb.fit_transform(df['sentiment_review'])
print(sentiment_data.shape)

(5401, 1)


**Split the sentiment tdata**



In [276]:
#Spliting the sentiment data
train_sentiments=sentiment_data[:4000]
test_sentiments=sentiment_data[4000:]
print(train_sentiments)
print(test_sentiments)

[[1]
 [1]
 [1]
 ...
 [1]
 [1]
 [1]]
[[1]
 [1]
 [0]
 ...
 [1]
 [1]
 [1]]


**Modelling the dataset**



Let us build logistic regression model for both bag of words and tfidf features



In [277]:
#training the model
lr=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
#Fitting the model for Bag of words
lr_bow=lr.fit(cv_train_reviews,train_sentiments)
print(lr_bow)
#Fitting the model for tfidf features
lr_tfidf=lr.fit(tv_train_reviews,train_sentiments)
print(lr_tfidf)

LogisticRegression(C=1, max_iter=500, random_state=42)
LogisticRegression(C=1, max_iter=500, random_state=42)


Logistic regression model performane on test dataset



In [278]:
#Predicting the model for bag of words
lr_bow_predict=lr.predict(cv_test_reviews)
print(lr_bow_predict)
##Predicting the model for tfidf features
lr_tfidf_predict=lr.predict(tv_test_reviews)
print(lr_tfidf_predict)

[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]


**Accuracy of the model**



In [279]:
#Accuracy score for bag of words
lr_bow_score=accuracy_score(test_sentiments,lr_bow_predict)
print("lr_bow_score :",lr_bow_score)
#Accuracy score for tfidf features
lr_tfidf_score=accuracy_score(test_sentiments,lr_tfidf_predict)
print("lr_tfidf_score :",lr_tfidf_score)

lr_bow_score : 0.9550321199143469
lr_tfidf_score : 0.9550321199143469


In [280]:
#Classification report for bag of words 
lr_bow_report=classification_report(test_sentiments,lr_bow_predict,target_names=['Positive','Negative'])
print(lr_bow_report)

#Classification report for tfidf features
lr_tfidf_report=classification_report(test_sentiments,lr_tfidf_predict,target_names=['Positive','Negative'])
print(lr_tfidf_report)

              precision    recall  f1-score   support

    Positive       0.00      0.00      0.00        63
    Negative       0.96      1.00      0.98      1338

    accuracy                           0.96      1401
   macro avg       0.48      0.50      0.49      1401
weighted avg       0.91      0.96      0.93      1401

              precision    recall  f1-score   support

    Positive       0.00      0.00      0.00        63
    Negative       0.96      1.00      0.98      1338

    accuracy                           0.96      1401
   macro avg       0.48      0.50      0.49      1401
weighted avg       0.91      0.96      0.93      1401



Confusion matrix



In [281]:
#confusion matrix for bag of words
cm_bow=confusion_matrix(test_sentiments,lr_bow_predict,labels=[1,0])
print(cm_bow)
#confusion matrix for tfidf features
cm_tfidf=confusion_matrix(test_sentiments,lr_tfidf_predict,labels=[1,0])
print(cm_tfidf)

[[1338    0]
 [  63    0]]
[[1338    0]
 [  63    0]]


Stochastic gradient descent or Linear support vector machines for bag of words and tfidf features



In [282]:
#training the linear svm
svm=SGDClassifier(loss='hinge',max_iter=500,random_state=42)
#fitting the svm for bag of words
svm_bow=svm.fit(cv_train_reviews,train_sentiments)
print(svm_bow)
#fitting the svm for tfidf features
svm_tfidf=svm.fit(tv_train_reviews,train_sentiments)
print(svm_tfidf)

SGDClassifier(max_iter=500, random_state=42)
SGDClassifier(max_iter=500, random_state=42)


Model performance on test data



In [283]:
#Predicting the model for bag of words
svm_bow_predict=svm.predict(cv_test_reviews)
print(svm_bow_predict)
#Predicting the model for tfidf features
svm_tfidf_predict=svm.predict(tv_test_reviews)
print(svm_tfidf_predict)

[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]


Accuracy of the model



In [284]:
#Accuracy score for bag of words
svm_bow_score=accuracy_score(test_sentiments,svm_bow_predict)
print("svm_bow_score :",svm_bow_score)
#Accuracy score for tfidf features
svm_tfidf_score=accuracy_score(test_sentiments,svm_tfidf_predict)
print("svm_tfidf_score :",svm_tfidf_score)

svm_bow_score : 0.953604568165596
svm_tfidf_score : 0.9550321199143469


Print the classification report



In [285]:
#Classification report for bag of words 
svm_bow_report=classification_report(test_sentiments,svm_bow_predict,target_names=['Positive','Negative'])
print(svm_bow_report)
#Classification report for tfidf features
svm_tfidf_report=classification_report(test_sentiments,svm_tfidf_predict,target_names=['Positive','Negative'])
print(svm_tfidf_report)

              precision    recall  f1-score   support

    Positive       0.25      0.02      0.03        63
    Negative       0.96      1.00      0.98      1338

    accuracy                           0.95      1401
   macro avg       0.60      0.51      0.50      1401
weighted avg       0.92      0.95      0.93      1401

              precision    recall  f1-score   support

    Positive       0.00      0.00      0.00        63
    Negative       0.96      1.00      0.98      1338

    accuracy                           0.96      1401
   macro avg       0.48      0.50      0.49      1401
weighted avg       0.91      0.96      0.93      1401



Plot the confusion matrix



In [286]:
#confusion matrix for bag of words
cm_bow=confusion_matrix(test_sentiments,svm_bow_predict,labels=[1,0])
print(cm_bow)
#confusion matrix for tfidf features
cm_tfidf=confusion_matrix(test_sentiments,svm_tfidf_predict,labels=[1,0])
print(cm_tfidf)

[[1335    3]
 [  62    1]]
[[1338    0]
 [  63    0]]


Multinomial Naive Bayes for bag of words and tfidf features



In [287]:
#training the model
mnb=MultinomialNB()
#fitting the svm for bag of words
mnb_bow=mnb.fit(cv_train_reviews,train_sentiments)
print(mnb_bow)
#fitting the svm for tfidf features
mnb_tfidf=mnb.fit(tv_train_reviews,train_sentiments)
print(mnb_tfidf)

MultinomialNB()
MultinomialNB()


Model performance on test data



In [288]:
#Predicting the model for bag of words
mnb_bow_predict=mnb.predict(cv_test_reviews)
print(mnb_bow_predict)
#Predicting the model for tfidf features
mnb_tfidf_predict=mnb.predict(tv_test_reviews)
print(mnb_tfidf_predict)

[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]


In [289]:
#Accuracy of the model
#Accuracy score for bag of words
mnb_bow_score=accuracy_score(test_sentiments,mnb_bow_predict)
print("mnb_bow_score :",mnb_bow_score)
#Accuracy score for tfidf features
mnb_tfidf_score=accuracy_score(test_sentiments,mnb_tfidf_predict)
print("mnb_tfidf_score :",mnb_tfidf_score)


mnb_bow_score : 0.9550321199143469
mnb_tfidf_score : 0.9550321199143469


In [290]:
#Classification report for bag of words 
mnb_bow_report=classification_report(test_sentiments,mnb_bow_predict,target_names=['positive','negative'])
print(mnb_bow_report)
#Classification report for tfidf features
mnb_tfidf_report=classification_report(test_sentiments,mnb_tfidf_predict,target_names=['positive','negative'])
print(mnb_tfidf_report)

              precision    recall  f1-score   support

    positive       0.00      0.00      0.00        63
    negative       0.96      1.00      0.98      1338

    accuracy                           0.96      1401
   macro avg       0.48      0.50      0.49      1401
weighted avg       0.91      0.96      0.93      1401

              precision    recall  f1-score   support

    positive       0.00      0.00      0.00        63
    negative       0.96      1.00      0.98      1338

    accuracy                           0.96      1401
   macro avg       0.48      0.50      0.49      1401
weighted avg       0.91      0.96      0.93      1401



In [291]:
#confusion matrix for bag of words
cm_bow=confusion_matrix(test_sentiments,mnb_bow_predict,labels=[1,0])
print(cm_bow)
#confusion matrix for tfidf features
cm_tfidf=confusion_matrix(test_sentiments,mnb_tfidf_predict,labels=[1,0])
print(cm_tfidf)

[[1338    0]
 [  63    0]]
[[1338    0]
 [  63    0]]
